In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean

# Example data (replace with your dataset)
data = {
    "user_id": [1, 2, 3, 4],
    "session_duration": [120, 300, 50, 400],
    "session_frequency": [5, 10, 2, 15],
    "total_traffic": [1000, 5000, 200, 8000]
}
df = pd.DataFrame(data)

# Normalize metrics
scaler = MinMaxScaler()
df[["session_duration", "session_frequency", "total_traffic"]] = scaler.fit_transform(
    df[["session_duration", "session_frequency", "total_traffic"]]
)

# Compute engagement and experience scores
df["engagement_score"] = df.apply(
    lambda row: euclidean([1, 1, 1], [row["session_duration"], row["session_frequency"], row["total_traffic"]]),
    axis=1
)
df["experience_score"] = 1 - df["engagement_score"]  # Example calculation; adjust as needed
df["satisfaction_score"] = (df["engagement_score"] + df["experience_score"]) / 2
print(df)


In [ ]:
from sklearn.cluster import KMeans

# Prepare clustering data
clustering_data = df[["engagement_score", "experience_score"]]
kmeans = KMeans(n_clusters=2, random_state=42)
df["satisfaction_cluster"] = kmeans.fit_predict(clustering_data)

# Aggregate scores per cluster
cluster_aggregates = df.groupby("satisfaction_cluster")[["satisfaction_score", "experience_score"]].mean()
print("Cluster Aggregates:")
print(cluster_aggregates)


In [ ]:
import mysql.connector

# Database connection details
db_config = {
    "user": "your_username",
    "password": "your_password",
    "host": "localhost",
    "database": "your_database"
}

# Connect to the database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Create table if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS satisfaction_analysis (
    user_id INT PRIMARY KEY,
    engagement_score FLOAT,
    experience_score FLOAT,
    satisfaction_score FLOAT,
    satisfaction_cluster INT
)
"""
cursor.execute(create_table_query)

# Insert data
for _, row in df.iterrows():
    insert_query = """
    REPLACE INTO satisfaction_analysis (user_id, engagement_score, experience_score, satisfaction_score, satisfaction_cluster)
    VALUES (%s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        row["user_id"],
        row["engagement_score"],
        row["experience_score"],
        row["satisfaction_score"],
        row["satisfaction_cluster"]
    ))

# Commit and close connection
connection.commit()
cursor.close()
connection.close()
print("Data exported to MySQL database.")